In [1]:
import os
import collections
import pandas as pd
from typing import List, Tuple, Dict
import re

In [2]:
PARQUET_PATH = "../Assignment_1/tokenized_hi.parquet"
MERGE_STEPS = 32000
VOCAB_SIZE = 32000
SAVE_DIR = "."

In [3]:
os.makedirs(SAVE_DIR, exist_ok=True)

In [4]:
df = pd.read_parquet(PARQUET_PATH)
print('\nColumns:', list(df.columns))
print('\nDataframe head:')
print(df.head(5))


Columns: ['paragraph_id', 'sentences']

Dataframe head:
   paragraph_id                                          sentences
0             0  [{'text': 'लोगों को बिलों संबंधी सुविधा देना ह...
1             1  [{'text': 'इनेलो 1987 में उस वक्त ऐसे ही दोराह...
2             2  [{'text': 'जहां आई थी तबाही उस घाटी क्षेत्र मे...
3             3  [{'text': 'इसके बाद केंद्र की ओर से प्रदेश सरक...
4             4  [{'text': 'यह पूछने पर कि इस बड़े मैच से पहले ...


In [5]:
TEXT_COL = "sentences"

In [6]:
import numpy as np

def extract_sentences_from_parquet(df):
    sentences = []
    for entry in df["sentences"]:
        # Case 1: numpy array of dicts
        if isinstance(entry, np.ndarray):
            for e in entry:
                if isinstance(e, dict) and 'text' in e:
                    text = e['text']
                    tokens = text.strip().split()
                    sentences.append(tokens)
        # Case 2: single dict
        elif isinstance(entry, dict) and 'text' in entry:
            text = entry['text']
            tokens = text.strip().split()
            sentences.append(tokens)
        # Case 3: list of dicts
        elif isinstance(entry, list):
            for e in entry:
                if isinstance(e, dict) and 'text' in e:
                    text = e['text']
                    tokens = text.strip().split()
                    sentences.append(tokens)
    return sentences

In [7]:
sentences = extract_sentences_from_parquet(df)
print(f"Loaded {len(sentences)} sentences. Example: {sentences[:2]}")

Loaded 3263994 sentences. Example: [['लोगों', 'को', 'बिलों', 'संबंधी', 'सुविधा', 'देना', 'ही', 'उनका', 'काम'], ['इनेलो', '1987', 'में', 'उस', 'वक्त', 'ऐसे', 'ही', 'दोराहे', 'पर', 'खड़ी', 'थी,', 'जब', 'पूर्व', 'उपप्रधानमंत्री', 'देवीलाल', 'ने', 'अपने', 'पुत्र', 'ओमप्रकाश', 'चौटाला', 'को', 'अपना', 'राजनीतिक', 'उत्तराधिकारी', 'घोषित', 'किया', 'था।']]


In [8]:
word_freq = collections.Counter()
for sent in sentences:
    for word in sent:
        word_freq[word] += 1

print(f"Unique words: {len(word_freq)}")
print("Most common:", word_freq.most_common(10))

Unique words: 1032539
Most common: [('के', 2377835), ('में', 1812145), ('की', 1429926), ('को', 1113308), ('से', 1051337), ('ने', 811565), ('का', 785793), ('और', 759579), ('है।', 751002), ('पर', 664240)]


In [9]:
#prepare the initial vocabulary
vocab = {' '.join(list(word)) + ' </w>': freq for word, freq in word_freq.items()}
print("Initial vocab size:", len(vocab))

Initial vocab size: 1032539


In [10]:
def get_stats(vocab):
    """Count frequency of all adjacent symbol pairs in vocab."""
    pairs = collections.Counter()
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i+1])] += freq
    return pairs

In [11]:
def merge_vocab(pair, v_in):
    """Merge the most frequent pair in vocab."""
    v_out = {}
    bigram = re.escape(' '.join(pair))
    pattern = re.compile(r'(?<!\\S)' + bigram + r'(?!\\S)')
    for word in v_in:
        new_word = pattern.sub(''.join(pair), word)
        v_out[new_word] = v_in[word]
    return v_out

In [12]:
for i in range(MERGE_STEPS):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    if (i+1) % 1000 == 0:
        print(f"Step {i+1}: merged {best}")

Step 1000: merged ('उप', 'ल')
Step 2000: merged ('य', 'द</w>')
Step 3000: merged ('ड', 'न</w>')
Step 4000: merged ('घा', 'टी</w>')
Step 5000: merged ('जाये', 'गा।</w>')
Step 6000: merged ('फ़', 'ि')
Step 7000: merged ('वर्', 'जन</w>')
Step 8000: merged ('झा', ',</w>')
Step 9000: merged ('सचि', 'व,</w>')


: 

In [ ]:
bpe_vocab = set()
for word in vocab:
    bpe_vocab.update(word.split())

with open("bpe_vocab.txt", "w", encoding="utf-8") as f:
    for token in sorted(bpe_vocab):
        f.write(token + "\\n")

print("BPE vocabulary saved at bpe_vocab.txt")

In [ ]:
def encode_bpe_word(word, merges):
    word = list(word) + ['</w>']
    pairs = [(word[i], word[i+1]) for i in range(len(word)-1)]
    merge_set = set(merges)
    while True:
        bigrams = [(a,b) for (a,b) in pairs if (a,b) in merge_set]
        if not bigrams:
            break
        a,b = bigrams[0]
        new_word = []
        i = 0
        while i < len(word):
            if i < len(word)-1 and word[i]==a and word[i+1]==b:
                new_word.append(a+b)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        word = new_word
        if len(word) == 1:
            break
        pairs = [(word[i], word[i+1]) for i in range(len(word)-1)]
    return word

print("Example BPE tokens:", encode_bpe_word("सुविधा", [("स","ु"),("वि","धा")]))

In [ ]:
def train_wordpiece(corpus, vocab_size):
    vocab = collections.Counter()
    for sent in corpus:
        for word in sent:
            vocab[word] += 1

    # Start with all characters
    wp_vocab = set()
    for word in vocab:
        wp_vocab.update(word)

    wp_vocab = {ch: i for i, ch in enumerate(sorted(wp_vocab))}

    while len(wp_vocab) < vocab_size:
        pairs = collections.Counter()
        for word, freq in vocab.items():
            chars = list(word)
            for i in range(len(chars) - 1):
                pairs[(chars[i], chars[i+1])] += freq
        if not pairs:
            break
        best = max(pairs, key=pairs.get)
        new_token = best[0] + best[1]
        wp_vocab[new_token] = len(wp_vocab)

        new_vocab = collections.Counter()
        for word, freq in vocab.items():
            new_word = word.replace(new_token, new_token)
            new_vocab[new_word] += freq
        vocab = new_vocab

    return wp_vocab

wp_vocab = train_wordpiece(sentences, 32000)
print("WordPiece vocab size:", len(wp_vocab))

In [ ]:
with open("wp_vocab.txt", "w", encoding="utf-8") as f:
    for token in wp_vocab:
        f.write(token + "\\n")

print("WordPiece vocabulary saved at wp_vocab.txt")

In [ ]:
def encode_word_wordpiece(word, vocab):
    output_tokens = []
    start = 0
    while start < len(word):
        end = len(word)
        sub = None
        while start < end:
            substr = word[start:end]
            if start > 0:
                substr = '##' + substr
            if substr in vocab:
                sub = substr
                break
            end -= 1
        if sub is None:
            output_tokens.append('[UNK]')
            start += 1
        else:
            output_tokens.append(sub)
            start = end
    return output_tokens

print("Example WordPiece tokens:", encode_word_wordpiece("सुविधा", wp_vocab))